In [1]:
import codecs
import os
from bs4 import BeautifulSoup
import cssutils
import operator
import logging
cssutils.log.setLevel(logging.CRITICAL)
from selenium import webdriver
import time
from operator import itemgetter

In [73]:
options = webdriver.FirefoxOptions()
options.add_argument("--headless")
options.add_argument("--disable-gpu")

In [2]:
browser = webdriver.Firefox(chrome_options=options)

In [3]:
def getBrowser(file_name):
    html_file = os.path.join(os.getcwd(), 'pdf/html', file_name)
    browser.get("file:///" + html_file)
    return browser

In [5]:
browser = getBrowser('20971.html')

In [35]:
bottom_limit = (930, 120)

class_list = []
for page_class in page_list:
    for e in browser.find_element_by_class_name(page_class).find_elements_by_xpath('.//div'):
        bottom_px = e.value_of_css_property('bottom')
        bottom = float(bottom_px.replace('px', ''))
        if ((bottom > bottom_limit[0]) | (bottom < bottom_limit[1])):
            browser.execute_script("""
            var element = arguments[0];
            element.parentNode.removeChild(element);
            """, e)

In [43]:
ff_list = []
fs_list = []
for e in browser.find_element_by_class_name(page_list[0]).find_elements_by_xpath('.//div'):
    class_list = e.get_attribute('class').split(' ')
    for c in class_list:
        if 'ff' in c:
            ff_list.append(c)
        if 'fs' in c:
            fs_list.append(c)

In [46]:
ff_frequent = max(set(ff_list), key = ff_list.count)
fs_frequent = max(set(fs_list), key = fs_list.count) 

In [53]:
fs_frequent_size = float(browser.find_element_by_class_name(fs_frequent).value_of_css_property('font-size').replace('px', ''))

In [54]:
for page_class in page_list:
    for e in browser.find_element_by_class_name(page_class).find_elements_by_xpath('.//div'):
        fs_px = e.value_of_css_property('font-size')
        fs = float(fs_px.replace('px', ''))
        if fs < fs_frequent_size:
            browser.execute_script("""
            var element = arguments[0];
            element.parentNode.removeChild(element);
            """, e)

In [67]:
last_page_no = browser.find_elements_by_xpath("//*[text()[contains(.,'판사')]]")[-1].find_element_by_xpath('..').find_element_by_xpath('..').get_attribute('data-page-no')

In [64]:
def getPNList(browser):
    page_no_list = []
    for e in browser.find_element_by_id('page-container').find_elements_by_xpath('*'):
        page_id = e.get_attribute('data-page-no')
        page_no_list.append(page_id)
    return page_no_list

In [69]:
page_no_list = getPNList(browser)
remove_page_list = page_no_list[page_no_list.index(last_page_no)+1:]

In [70]:
for e in browser.find_element_by_id('page-container').find_elements_by_xpath('*'):
    page_id = e.get_attribute('data-page-no')
    if page_id in remove_page_list:
        browser.execute_script("""
        var element = arguments[0];
        element.parentNode.removeChild(element);
        """, e)

In [71]:
for page_class in page_list:
    for e in browser.find_element_by_class_name(page_class).find_elements_by_xpath('.//img'):
        browser.execute_script("""
        var element = arguments[0];
        element.parentNode.removeChild(element);
        """, e)

NoSuchElementException: Message: Unable to locate element: .pc1a


In [72]:
import json, base64

In [80]:
def send_devtools(driver, cmd, params={}):
    resource = "/session/%s/chromium/send_command_and_get_result" % driver.session_id
    url = driver.command_executor._url + resource
    body = json.dumps({'cmd': cmd, 'params': params})
    response = driver.command_executor._request('POST', url, body)
    if response['status']:
        raise Exception(response.get('value'))
    return response.get('value')

def save_as_pdf(driver, path, options={}):    
    result = send_devtools(driver, "Page.printToPDF", options)
    if (result is not None):
        with open(path, 'wb') as file:
            file.write(base64.b64decode(result['data']))
        return True
    else:
        return False

In [81]:
save_as_pdf(browser, r'page.pdf', { 'landscape': False })

Exception: HTTP method not allowed

In [24]:
for page_class in page_list:
    for e in browser.find_element_by_class_name(page_class).find_elements_by_xpath('.//img'):
        print(e)

<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="7954c4ef-896d-4e7d-aebb-db603203ca13", element="5976a22c-c8e3-45ed-b37a-56fa26e3e70f")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="7954c4ef-896d-4e7d-aebb-db603203ca13", element="f3fb6f81-b7a5-4885-9976-d3d24bccf494")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="7954c4ef-896d-4e7d-aebb-db603203ca13", element="0169c0fb-0b8a-4c51-b6ae-1d1bcf12ea65")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="7954c4ef-896d-4e7d-aebb-db603203ca13", element="42ecc0b8-5cae-426a-ac59-8de39f9a44d6")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="7954c4ef-896d-4e7d-aebb-db603203ca13", element="47822136-7557-40b5-95aa-30bdddfbec60")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="7954c4ef-896d-4e7d-aebb-db603203ca13", element="092ec1c3-639e-41b9-aab0-f7e183f45ddf")>
<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="795

In [21]:
for page_class in page_list:
    for e in browser.find_element_by_class_name(page_class).find_elements_by_xpath('.//div'):
        print(e.get_attribute('textContent'))

- 1 -
울  산  지  방  법  원
제 1  2  민 사 부
판           결
사       건 2015가합23143  임금
원       고 A 외 21인
원고들 소송대리인 법무법인 @@
담당변호사 **
피       고 학교법인 ○○공업학원
이사장 B
소송대리인 법무법인 ##
담당변호사 %%
변 론 종 결 2019. 5. 29.
판 결 선 고 2019. 7. 10.
주       문
1. 피고는 원고들에게 별지 1 '인용 금액표‘ 중 ’인용 금액‘란 기재 각 돈 및 위 각 돈
에 대하여 2014. 10. 1.부터 2019. 7. 10.까지는 연 6%, 그다음 날부터 다 갚는 날까
지는 연 15%의 각 비율로 계산한 돈을 지급하라. 
2. 원고들의 나머지 청구를 각 기각한다.  
- 2 -
3. 소송비용은 피고가 부담한다.
4. 제1항은 가집행할 수 있다. 
청 구 취 지
피고는 원고들에게 별지 1 ‘인용 금액표’ 중 ‘인용 금액’란 기재 각 돈 및 위 각 돈에 
대하여 2014. 10.  1.부터 이  사건 청구취지변경신청서 부본 송달일까지는 연 6%,  그 
다음 날부터 2015. 9. 30.까지는 연 20%, 2015. 10. 1.부터 다 갚는 날까지는 연 15%
의 각 비율로 계산한 돈을 각 지급하라.
이       유
1. 기초사실 
  가. 원고들은 ○○대학교 병원에서 간호사, 임상병리사, 방사선사 및 어린이집 교사 
등으로 근무하고 있거나 근무하다가 퇴직한 자들이고, 피고는 ○○대병원을 운영하는 
학교법인이다. 
  나. 피고는 원고들을 비롯한 소속 근로자들에게 매년 700%의 상여금(= 짝수월 상여 
500% + 명절상여 200%) 및 150%의 성과금을 지급해 왔고, 2004. 7. 1.부터 근로기준
법 개정에 따른 휴가감소분에 대하여 휴가보전수당을 지급하여 왔다. 또한, 단체협약에
서 정해진 근로시간을 넘는 연장, 야간, 휴일근로에 관하여 단체협약에서 정해진 대로
의  연장ㆍ야간ㆍ휴일근로수당을  지급하였으며,  단체협약에서  정

12월  -  50%
2월  -  100% 
       
       
(
(
단
단
, 
, 
α
α
분은 
분은 
병원경영실적을 
병원경영실적을 
감안하여 
감안하여 
노사간 
노사간 
협의에 
협의에 
의하여 
의하여 
정한다
정한다
)
)
    2.  성과금  지급시  회계연도  기준(매년  3월  1일  ~  익년  2월  28일)  1년간의  근태사항을  반영한다. 
        (1)  지각,  조퇴:  지각,  조퇴  3회는  유결  1일로  간주
        (2)  유          결:  1일당  3.3％  감액
        (3)  무          결:  유결의  1.5배로  간주  1일당  4.95%  감액   
(별표  #1)
상여금지급기준 
[인정 근거] 다툼 없는 사실, 을 제1 내지 3호증의 각 기재, 변론 전체의 취지 
2. 당사자들 주장의 요지
1)  갑  제26호증(2014년도  임금  및  보충협약  합의서)의  기재에  의하면,  단체협약  제68조는  2014.  3.  1.부로  소급하여  상여금 
500% 및 격려금 400%를 지급하는 것으로 하면서 격려금은 그 지급일 현재 30일 이상 재직 중인 자에 한하여 지급하고 중도 
퇴사시에도 일할 계산하여 지급하지 않기로 개정되었으나, 이미 구체적으로 그 지급청구권이 발생한 임금(상여금 포함)은 근
로자의 사적 재산영역으로 옮겨져 근로자의 처분에 맡겨진 것이어서, 노동조합이 근로자들로부터 개별적인 동의나 수권을 받
지 않는 이상 사용자와 사이의 단체협약만으로 이에 대한 포기나 지급유예와 같은 처분행위를 할 수 없는바(대법원 2010. 1. 
28. 선고 2009다76317 판결 등 참조), 상여금 및 성과금이 통상임금에 해당함은 뒤에서 본 바와 같으므로 위 규정이 개정되
었다고 하여 2014. 3. 1.부로 소급하여 원고들의 상여금 및 성과금 중 일부의 통상임금성이 부정된다고 볼 것은 아니다.    
2) 이 사건 원고들과 관련된 부분만 인용한다. 
- 9 

- 17 -
수당을 시급으로 환산하면 별지 2 ‘원고별 청구금액 산정내역’ 중 ‘추가 통상시급’란 기
재 각 해당 금액인 사실, 위 원고들의 각 원고별 청구기간 내에 연장ㆍ야간ㆍ휴일 근
로시간 수와 미사용 연차일수가 별지 2 ‘원고별 청구금액 산정내역’ 중 ‘시간외 근로시
간 수’란 기재 시간 및 ‘연차일수’란 기재 연차일수인 사실은 당사자 사이에 다툼이 없
고, 앞서 본 바와 같이 미반영 추가 통상시급을 토대로 위 원고들에게 미지급된 연장
ㆍ야간ㆍ휴일근로수당(단체협약에 따른 중복 가산금 포함), 연차휴가수당의 합계가 별
지 2 ‘원고별 청구금액 산정내역’ 중 ‘청구금액’란 기재 각 금액의 합계인 사실 역시 다
툼이 없다. 
  다. 소결론
    따라서 특별한 사정이 없는 한 피고는 위 원고들에게 별지 1 ‘인용 금액표’ 중 ‘인
용 금액’란 기재 각 돈 및 이에 대하여 위 원고들이 구하는 바에 따라 2014. 10. 1.부
터 피고가 그 이행의무의 범위 및 존부에 관하여 항쟁함이 타당한 이 법원 판결선고일
인 2019. 7. 10.까지는 상법이 정한 연 6%, 그다음 날부터 다 갚는 날까지는 소송촉진 
등에 관한 특례법이 정한 연 15%의 각 비율로 계산한 지연손해금을 지급할 의무가 있
다.       
5. 원고 17 내지 21의 청구에 관한 판단 
  가. 미지급 임금 청구에 관한 판단 
    앞서 든 각 증거들에다가 갑 제15호증의 기재 및 변론 전체의 취지에 의하면, ㉠ 
피고  내  취업규칙은  ‘○○대학교  교원임용규정  및  병원  인사관리규정이  정하는  바에 
따라 병원에 채용되어 근로계약을 체결한 자’를 적용대상으로 밝히고 있을 뿐 위 원고
들과 같은 계약직의 경우 달리 구별을 두고 있지 않은 점, ㉡ 직원의 급여에 관하여는 
- 18 -
취업규칙에서 정한 것 외에는 보수규정에 따라야 하고, 수당의 종류, 지급기준, 지급방
법에 관하여서도 별도 보수규정 및 단체협약에 의하도록 하고 있는데, 피고 내 보수규
정 및 상여금 지급 규정 역시 

In [ ]:
element = driver.find_element_by_class_name('classname')
driver.execute_script("""
var element = arguments[0];
element.parentNode.removeChild(element);
""", element)

In [6]:
def getPCList(browser):
    page_class_list = []
    for e in browser.find_element_by_id('page-container').find_elements_by_xpath('*'):
        page_id = e.get_attribute('id')
        page_class = page_id.replace('pf', 'pc')
        page_class_list.append(page_class)
    return page_class_list

In [7]:
page_list = getPCList(browser)

In [8]:
page_list

['pc1',
 'pc2',
 'pc3',
 'pc4',
 'pc5',
 'pc6',
 'pc7',
 'pc8',
 'pc9',
 'pca',
 'pcb',
 'pcc',
 'pcd',
 'pce',
 'pcf',
 'pc10',
 'pc11',
 'pc12',
 'pc13',
 'pc14',
 'pc15',
 'pc16',
 'pc17',
 'pc18',
 'pc19',
 'pc1a',
 'pc1b',
 'pc1c',
 'pc1d',
 'pc1e',
 'pc1f',
 'pc20',
 'pc21',
 'pc22',
 'pc23',
 'pc24',
 'pc25',
 'pc26',
 'pc27',
 'pc28',
 'pc29',
 'pc2a',
 'pc2b',
 'pc2c',
 'pc2d',
 'pc2e',
 'pc2f']